# GBR_all_data_w_met_sddv4_h99_add_knn_xgboost

The following conditions apply:

 - env = biomass_zonal
 - data merged_slats_field_agb_dp1_start.csv

In [ ]:

%matplotlib inline


import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import ExtraTreesRegressor as etr
from sklearn.ensemble import GradientBoostingRegressor as gbr
from sklearn.ensemble import AdaBoostRegressor as abr
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.neighbors import KNeighborsRegressor as knn
from xgboost import XGBRegressor as xgboost
from sklearn.model_selection import train_test_split
import scipy.stats as sc
import textwrap

# stats module
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.metrics import explained_variance_score
%matplotlib inline

# import plotting and stats modules
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy
import pandas as pd

# Set option to display floating-point numbers without scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from bokeh.io import output_notebook, output_file
from bokeh.plotting import save # figure, show, 
#%matplotlib inline

# Bokeh Libraries
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, NumeralTickFormatter, HoverTool


### Import data

In [ ]:

# mdl = "RFR"
# model_name = "Random Forrest Regressor"


#mdl = "ABR"
#model_name = "AdaBoost Regressor"

mdl = "GBR"
model_name = "Gradient Boosting Regressor"

# mdl = "KNN"
# model_name = "K-Nearest Neighbors Regressor"

# mdl = "XGBR"
# model_name = "XGBoost Regressor"


rs = 0
drive = "D"
data_comp = f"{mdl}_dp1_dbi_si_dry_mask_density_sddv4_h99"

In [ ]:
csv_file = r"C:\Users\robot\projects\biomass\collated_zonal_stats\dry_mask\dp1_dbi_si_dry_mask_density.csv"

## Set output file locations

In [ ]:
output = r"C:\Users\robot\projects\biomass\model\{}".format(data_comp)
output_ = os.path.join(output, "plots")

In [ ]:
if not os.path.isdir(output):
    os.mkdir(output)
if not os.path.isdir(output_):
    os.mkdir(output_)
    
   

In [ ]:
def_path = os.path.join(output_, "DEF")    
if not os.path.isdir(def_path):
    os.mkdir(def_path)
    
r2_path = os.path.join(output_, "R2")    
if not os.path.isdir(r2_path):
    os.mkdir(r2_path)
    
mae_path = os.path.join(output_, "MAE")    
if not os.path.isdir(mae_path):
    os.mkdir(mae_path)
    
rmse_path = os.path.join(output_, "RMSE")    
if not os.path.isdir(rmse_path):
    os.mkdir(rmse_path)
    

In [ ]:
# read as dataframe and copy
df1 = pd.read_csv(csv_file, header=0) # the first row is read in as the header for you columns
print(df1.shape) # prints out the number of rows and columns in your csv file 
print(list(df1))
df1.shape
#df2 = df1.copy()

In [ ]:
# Check for null values in each column
columns_with_nulls = df1.columns[df1.isnull().any()]
columns_with_nulls

## Fill missing values

In [ ]:
# Fill missing values with the minimum value of each column
df1 = df1.apply(lambda col: col.fillna(col.min()), axis=0)

In [ ]:
# Check for null values in each column
columns_with_nulls = df1.columns[df1.isnull().any()]
columns_with_nulls

In [ ]:
df = df1.rename(columns={'bio_agb_kg1ha': 'target'})

In [ ]:
df

In [ ]:
columns_with_nulls = df.columns[df.isnull().any()]
columns_with_nulls

In [ ]:
print(df.columns.tolist())

## Main only

In [ ]:
#main major and h99 
var_ = data_comp

df_columns = list(df.columns)
keep = ['site_clean', "target", 
        #"mean", 
        "major", 
        "p99", 
        "GNDVI", "MSR", "NBR", "_NDVI", "CVI", "GDVI", "GSAVI",
        "NDGI","RI", "NDII", "MSAVI", "SAVI"
       'r32', 'r42', 'r43',
         'r52', 'r53', 'r54', 'r62', 'r63', 'r64', 'r65',
         #'dr_', 'ma_', 'tx_', 'tn_', 'rx_', 'rn_'
        ]
header = [ele for ele in df_columns for x in keep if x in ele]
df2 = df[header]
print(df2.shape)

In [ ]:
# drop fdc - no need for this data

# Identify columns that contain "fdc" in their column names
columns_to_drop = df2.columns[df2.columns.str.contains("fdc", case=False)]

# Drop these columns
df2 = df2.drop(columns=columns_to_drop)

In [ ]:
columns_to_drop

In [ ]:
df2.shape

In [ ]:
print(df2.columns.tolist())

In [ ]:
df_columns = list(df2.columns)
keep = ['major']
       
classified_cols = [ele for ele in df_columns for x in keep if x in ele]
print(classified_cols)

In [ ]:
# df2 = df2[['site_clean', 'target', 'dbifmdry_r42', 'dbifmdry_r43', 'dbifmdry_r52', 'dbifmdry_r53', 'dbifmdry_r54', 'dbifmdry_r62', 'dbifmdry_r63', 'dbifmdry_r64', 'dbifmdry_r65', 'dbifmdry_NDVI',
# 
#  ]]
# print(df2.columns.tolist())
# var_ = "_ratio_NDVI_only_test"

In [ ]:
df2.columns.tolist()

### Logical selection

In [ ]:
# Check for duplicate columns and print them
duplicate_columns = df2.columns[df2.columns.duplicated()]

if duplicate_columns.any():
    print("Duplicate columns found:")
    for col in duplicate_columns:
        print(col)
else:
    print("No duplicate columns found.")

## Remove site values which seem like outliers

Detect outliers using Z score on all columns including Target


In [ ]:
df2.info()

## Detect Z score outliers on all columns except target

In [ ]:
import pandas as pd
from scipy.stats import zscore
import numpy as np

# Function to detect outliers using Z-score
def detect_outliers(df, target_column):
    # Select only numeric columns except the target column
    numeric_df = df.select_dtypes(include=[np.number]).drop(columns=[target_column])
    
    # Calculate the Z-scores
    z_scores = np.abs(zscore(numeric_df))
    
    # Identify rows with Z-scores greater than 3 in any column
    outliers = (z_scores > 4).any(axis=1)
    
    return outliers

# Assuming df2 is your DataFrame with mixed data types and 'target' is the target column
df = df2.copy()

# Define the target column
target_column = 'target'  # Replace 'target' with the name of your target column

# Detect outliers
outliers = detect_outliers(df, target_column)

# Print the detected outliers
print("Detected Outliers:")
print(df[outliers])

# If you want to highlight these outliers in the original DataFrame
df_highlighted = df.copy()

for col in df.select_dtypes(include=[np.number]).drop(columns=[target_column]).columns:
    df_highlighted[col + '_outlier'] = np.where(outliers, 'Outlier', 'Normal')

print("Original DataFrame with Outliers Highlighted:")
print(df_highlighted)

output_ = os.path.join(output, f"dry_mask_{var_}_not_target_outlier.csv")
df_highlighted.to_csv(output_, index=False)

In [ ]:
# Function to detect outliers using Z-score
def detect_outliers(df):
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calculate the Z-scores
    z_scores = np.abs(zscore(numeric_df))
    
    # Identify rows with Z-scores greater than 3 in any column
    outliers = (z_scores > 4).any(axis=1)
    
    return outliers

# Assuming df2 is your DataFrame with mixed data types
df = df2.copy()

# Detect outliers
outliers = detect_outliers(df)

# Print the detected outliers
print("Detected Outliers:")
print(df[outliers].site_clean)

# Drop rows that contain outliers
df_cleaned = df[~outliers]

print("Cleaned DataFrame (without outliers):")
df_cleaned

In [ ]:
df_cleaned

In [ ]:
df_cleaned.shape

In [ ]:
df2 = df_cleaned

### Set variables to plot

In [ ]:
# predicted value is x
value_x = 'target'
# variable is y
value_y = "b1_wfp_mean"
value_a = 'b2_dp1fm_dry_mean'
value_b = 'b1_h99_mean'


site = 'site_clean'


In [ ]:
value_y

In [ ]:
df2.shape

In [ ]:
output_int = os.path.join(output, "inter")
if not os.path.isdir(output_int):
    os.mkdir(output_int)


In [ ]:
def save_fig(value_x, value_y):
    # Output to file
    output_file(os.path.join(output_int,'all_sites_{0}_{1}.html'.format(value_x, value_y)),
                title='Relationship between {0} and {1}'.format(value_x.replace("_", " "), value_y.replace("_", " ")))


    #Specify the selection tools to be made available
    select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'zoom_in', 'zoom_out', 'wheel_zoom', 'reset']

    #print(test)
    # Format the tooltip
    tooltips = [
                ('Site', '@site_clean'),
                (value_x, '@{0}'.format(value_x)),
                (value_y, '@{0}'.format(value_y)),   
                (value_a, '@{0}'.format(value_a)),
                (value_b, '@{0}'.format(value_b)) 
               ]

    # Create the figure
    fig = figure(plot_height=400,
                 plot_width=1500,
                 y_axis_label= value_y.replace("_", " "), 
                 x_axis_label= value_x.replace("_", " "),
                 title='Relationship between {0} and {1}'.format(value_x.replace("_", " "), value_y.replace("_", " ")),
                 toolbar_location='below',
                 tools=select_tools)

    # # Format the y-axis tick label
    fig.yaxis[0].formatter = NumeralTickFormatter(format='0')

    # Add square representing each site
    fig.square(x= value_x,
               y= value_y,
               source=df2.round(4),
               size=5,
               color='royalblue',
               selection_color='deepskyblue',
               nonselection_color='lightgray',
               nonselection_alpha=0.3)

    # Add the HoverTool to the figure
    fig.add_tools(HoverTool(tooltips=tooltips))

    # Visualize
    save(fig)

In [ ]:
df2.shape

In [ ]:
column_list = df2.columns.to_list()
y_list = column_list[3:]
value_x = column_list[1:2][0]


for i in y_list:
    value_y = i
    save_fig(value_x, value_y)
    

In [ ]:
print(column_list[3:])
print(column_list[1:2])

### One-Hot Encoding (for Tree-based Models)

In [ ]:
# add columns that contain class data n17
classified_cols

In [ ]:
df2.columns

In [ ]:
for i in classified_cols:
    df2 = pd.get_dummies(df2, columns=[i], prefix=f'{i}_en')

In [ ]:
#df_encoded
df2

## Choose which data set to run the models from

In [ ]:
df_ml = df2

In [ ]:
# drop some of the unwanted values
df_ml.drop(['site_clean',], axis=1, inplace=True)
#df_ml.drop(['fpca2_imdate'], axis=1, inplace=True)

In [ ]:
print(list(df_ml))

In [ ]:
df_ml.shape

In [ ]:
df_ml.info()

## Reduce level of 0 values is this cell needed?

## Drop AGB numbers which are low

In [ ]:
# df2 = df2[df2['target']>0.0]
# #df2 = df2[df2['target']>1000.0]
# df2.shape

In [ ]:
# columns_with_nulls = df2.columns[df2.isnull().any()]
# columns_with_nulls

## Drop AGB numbers which are high

In [ ]:
# # drop the 7 tern sites that appear to be outliers
# df2 =df2[df2['target'] <= 40000]
# df2.to_csv(os.path.join(output, "{0}_lt_40000.csv".format('target')))
# #df2 =df2[df2['target'] > 40000]
# #df2.value_counts(['site_clean', value_x, value_y])

In [ ]:
df_ml.shape

In [ ]:
#no removal
out_df = df_ml
samp = "all_data"

# # due to the number of field sites with no basal collected data is stratified
# out_df = df_ml[df_ml['target']>0.0]
# samp = "no0"
# out_df.to_csv(os.path.join(output, "more_than_0kgha.csv"))
# out_df = df_ml[df_ml['target']>=10.0]
# samp="more_t_10"
# out_df.to_csv(os.path.join(output, "more_than_10kgha.csv"))
# 
# # create a random selection of 0 - based on sample size
# sample_size = 3
# no_0_df = df_ml[df_ml['target']>0.0]
# samp = "no0"
# agb_0 = df_ml[df_ml['bio_agb_kg1ha']==0.0].sample(sample_size)
# out_df = pd.concat([no_0_df, agb_0])
# out_df.to_csv(os.path.join(output, "ml_df_0_sample_{0}kgha.csv".format(str(sample_size))))
# samp = f"some0_{sample_size}"
# 
# 
# out_df =df_ml[df_ml['target'] <= 40000]
# out_df.to_csv(os.path.join(output, "less_than_40000kgha.csv"))
# samp = "less_t_40"

In [ ]:
test = out_df[out_df['target']>40000.0]
test.shape

## Select the dataset to run the models from

In [ ]:
# All variables
df_ml = out_df

# select variables
#df = select_df
df_ml.shape

 Define if you are using all variabes or selected variables

In [ ]:
df_ml.info()

In [ ]:
df_ml.dropna(inplace=True)

In [ ]:
df_ml.info()

In [ ]:
df_ml.describe()

# Scatter plots with error

In [ ]:
#df_ml.to_csv(, index=False)
df_ml.to_csv(os.path.join(output, "{0}_{1}_{2}_ml_data.csv".format(var_, mdl, samp)), index=False)

In [ ]:
df_ml.describe()

In [ ]:
# Check for null values
null_values = df_ml.isnull().sum()

In [ ]:
null_values

In [ ]:
#df = df_ml

In [ ]:
# Function to check if a value is in scientific notation
def is_scientific_notation(value):
    try:
        float_value = float(value)
        return '{:e}'.format(float_value) == value.lower()
    except ValueError:
        return False

# Check for scientific notation in each cell
for column in df_ml.columns:
    for value in df_ml[column]:
        if is_scientific_notation(str(value)):
            print(f"Column {column}: {value} is in scientific notation")

In [ ]:
df = df_ml

def plot_with_mean_median(df, columns):
    for col in columns:
        plt.figure(figsize=(14, 10))

        # Top-left plot: Distribution with mean and median
        plt.subplot(2, 2, 1)
        sns.histplot(df[col], kde=True, color='blue', bins=30)
        
        # Calculate mean and median
        mean = df[col].mean()
        median = df[col].median()
        
        # Plot mean and median as dashed lines
        plt.axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean:.2f}')
        plt.axvline(median, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median:.2f}')
        
        plt.title(f'Distribution of {col}', fontsize=16)
        plt.xlabel(col, fontsize=14)
        plt.ylabel('Frequency', fontsize=14)
        plt.legend(fontsize=12)
        
        # Top-right plot: Regression plot
        value_x = 'target'
        C_value_x = "Target"
        value_y_loop = col

        plt.subplot(2, 2, 2)
        sns.regplot(data=df_ml, x=value_x, y=value_y_loop, line_kws={"color": "red"}, scatter_kws={'s': 50, 'alpha': 0.5, 'color': 'blue'})
        plt.xlabel(C_value_x, fontsize=14)
        plt.ylabel(value_y_loop, fontsize=14)
        plt.title(f"Regression of {C_value_x} vs. {value_y_loop}", fontsize=16)

        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(df_ml[value_x], df_ml[value_y_loop])

        # Annotate the stats
        plt.text(0.95, 0.95, f'slope: {slope:.2f}\nintercept: {intercept:.2f}\nr2: {r_value:.2f}\np-value: {p_value:.2f}\nstd err: {std_err:.2f}',
                 transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='right', fontsize=12)

        # Bottom-left plot: Residuals plot
        plt.subplot(2, 2, 3)
        sns.regplot(data=df_ml, x=value_x, y=value_y_loop, lowess=True, line_kws={"color": "green"}, scatter_kws={'s': 50, 'alpha': 0.5, 'color': 'blue'})
        plt.xlabel(C_value_x, fontsize=14)
        plt.ylabel("Error", fontsize=14)
        plt.title(f"Residuals of Regression: {C_value_x} vs. {value_y_loop}", fontsize=16)

        residuals = df_ml[value_y_loop] - (slope * df_ml[value_x] + intercept)

        # Compute additional residual statistics
        mean_res = np.mean(residuals)
        std_res = np.std(residuals)
        rmse_res = np.sqrt(np.mean(residuals**2))
        mae_res = np.mean(np.abs(residuals))
        r2_res = scipy.stats.linregress(df_ml[value_x], residuals)[2]**2

        # Annotate the residual stats
        plt.text(0.95, 0.95, f'Mean: {mean_res:.2f}\nStd: {std_res:.2f}\nRMSE: {rmse_res:.2f}\nMAE: {mae_res:.2f}\nr2: {r2_res:.2f}',
                 transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='right', fontsize=12)

        # Bottom-right plot: Q-Q plot
        plt.subplot(2, 2, 4)
        scipy.stats.probplot(residuals, dist="norm", plot=plt)
        plt.title(f"Q-Q Plot of Residuals: {C_value_x} vs. {value_y_loop}", fontsize=16)
        
        # Increase font size for Q-Q plot
        plt.xlabel('Theoretical Quantiles', fontsize=14)
        plt.ylabel('Sample Quantiles', fontsize=14)
        plt.tick_params(axis='both', which='major', labelsize=12)

        plt.tight_layout()

        # Save the plot
        plt_out = os.path.join(output, "plots", f"{var_}_{mdl}_{value_y_loop}_{rs}_combined_plot_tr.JPG")
        plt.savefig(plt_out, dpi=300)
        plt.show()
        print("plot: ", plt_out)
        plt.close()

# Assuming df_ml.columns[1:] contains the columns to be plotted
plot_with_mean_median(df, df_ml.columns[1:])

plt.close('all')

In [ ]:
df = df_ml

def plot_with_mean_median(df, columns):
    for col in columns:
        plt.figure(figsize=(14, 10))

        # Top-left plot: Distribution with mean and median
        plt.subplot(2, 2, 1)
        sns.histplot(df[col], kde=True, color='blue', bins=30)
        
        # Calculate mean and median
        mean = df[col].mean()
        median = df[col].median()
        
        # Plot mean and median as dashed lines
        plt.axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean:.2f}')
        plt.axvline(median, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median:.2f}')
        
        plt.title(f'Distribution of {col}', fontsize=16)
        plt.xlabel(col, fontsize=14)
        plt.ylabel('Frequency', fontsize=14)
        plt.legend(fontsize=12)
        
        # Top-right plot: Regression plot
        value_x = 'target'
        C_value_x = 'Target'
        value_y_loop = col

        plt.subplot(2, 2, 2)
        sns.regplot(data=df_ml, x=value_x, y=value_y_loop, line_kws={"color": "red"}, scatter_kws={'s': 50, 'alpha': 0.5, 'color': 'blue'})
        plt.xlabel(C_value_x, fontsize=14)
        plt.ylabel(value_y_loop, fontsize=14)
        plt.title(f"Regression of {C_value_x} vs. {value_y_loop}", fontsize=16)

        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(df_ml[value_x], df_ml[value_y_loop])

        # Annotate the stats
        plt.text(0.95, 0.05, f'slope: {slope:.2f}\nintercept: {intercept:.2f}\nr2: {r_value:.2f}\np-value: {p_value:.2f}\nstd err: {std_err:.2f}',
                 transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right', fontsize=12)

        # Bottom-left plot: Residuals plot
        plt.subplot(2, 2, 3)
        sns.regplot(data=df_ml, x=value_x, y=value_y_loop, lowess=True, line_kws={"color": "green"}, scatter_kws={'s': 50, 'alpha': 0.5, 'color': 'blue'})
        plt.xlabel(C_value_x, fontsize=14)
        plt.ylabel("Error", fontsize=14)
        plt.title(f"Residuals of Regression: {C_value_x} vs. {value_y_loop}", fontsize=16)

        residuals = df_ml[value_y_loop] - (slope * df_ml[value_x] + intercept)

        # Compute additional residual statistics
        mean_res = np.mean(residuals)
        std_res = np.std(residuals)
        rmse_res = np.sqrt(np.mean(residuals**2))
        mae_res = np.mean(np.abs(residuals))
        r2_res = scipy.stats.linregress(df_ml[value_x], residuals)[2]**2

        # Annotate the residual stats
        plt.text(0.95, 0.05, f'Mean: {mean_res:.2f}\nStd: {std_res:.2f}\nRMSE: {rmse_res:.2f}\nMAE: {mae_res:.2f}\nr2: {r2_res:.2f}',
                 transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right', fontsize=12)

        # Bottom-right plot: Q-Q plot
        plt.subplot(2, 2, 4)
        scipy.stats.probplot(residuals, dist="norm", plot=plt)
        plt.title(f"Q-Q Plot of Residuals: {C_value_x} vs. {value_y_loop}", fontsize=16)
        
        # Increase font size for Q-Q plot
        plt.xlabel('Theoretical Quantiles', fontsize=14)
        plt.ylabel('Sample Quantiles', fontsize=14)
        plt.tick_params(axis='both', which='major', labelsize=12)

        plt.tight_layout()

        # Save the plot
        plt_out = os.path.join(output, "plots", f"{var_}_{mdl}_{value_y_loop}_{rs}_combined_plot_br.JPG")
        plt.savefig(plt_out, dpi=300)
        #plt.show()
        print("plot: ", plt_out)
        plt.close()

# Assuming df_ml.columns[1:] contains the columns to be plotted
plot_with_mean_median(df, df_ml.columns[1:])

plt.close('all')

a# Model

### Split off validation test and train data

In [ ]:
value_x = "target"

In [ ]:
#select random state
#rs = 0

# randomly split data into train and test datasets, the user needs to define the variables 
xdata1 = df_ml.iloc[:, 1:].astype('float32')
ydata1 = df_ml[[value_x]].astype('float32')
ydata2 = ydata1.values
ydata = ydata2.ravel()
#y_data_float=ydata.astype("float32")
x_validation, x_remaining, y_validation, y_remaining = train_test_split(xdata1, ydata, train_size=0.20, random_state=rs)
x_train, x_test, y_train, y_test = train_test_split(x_remaining, y_remaining, train_size=0.70, random_state=rs)  
print(x_validation.shape, y_validation.shape)
print("remaining.....")
print(x_remaining.shape, y_remaining.shape)
print("-"*50)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

### Plot up Histograms for train and test datasets

In [ ]:
import numpy as np
# output = '.'  # Replace with your actual output directory
# var_ = 'example_var'
# mdl = 'example_model'
# rs = 'example_run'

# Create subplots: 1 row, 3 columns
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # Adjust figsize as needed

# Define the data and titles for each subplot
data = [y_train, y_test, y_validation]
titles = ['Training Data Distribution', 'Testing Data Distribution', 'Validation Data Distribution']

for i, (data_set, title) in enumerate(zip(data, titles)):
    mean = np.mean(data_set)
    median = np.median(data_set)
    
    # Plot histogram with seaborn
    sns.histplot(data_set, kde=True, bins=20, ax=axs[i], color='blue', alpha=0.7)
    
    # Plot mean and median lines
    axs[i].axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean:.2f}')
    axs[i].axvline(median, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median:.2f}')
    
    axs[i].set_title(title, fontsize=16)
    axs[i].set_xlabel('Target', fontsize=14)
    axs[i].set_ylabel('Frequency', fontsize=14)
    axs[i].legend(fontsize=12)

# Adjust layout
plt.tight_layout()

# Save the figure
out = os.path.join(output, "plots", f"{var_}_{mdl}_{rs}_train_test_validation_distribution.JPG")
plt.savefig(out, dpi=300)
plt.show()

print("saved to: ", out)

### Parameterise algorithm

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

if mdl == "RFR":
    from sklearn.ensemble import RandomForestRegressor as rfr

    # Define model
    model = rfr()
    
    # Define parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150, 200, 300],
        'max_depth': [None, 2, 3, 4, 5, 10, 20],
        'max_features': ['auto', 'sqrt'],
        'bootstrap': [True, False]
    }

elif mdl == "GBR":
    from sklearn.ensemble import GradientBoostingRegressor as gbr

    # Define model
    model = gbr()

    # Define parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150, 200, 300],
        'learning_rate': [0.01, 0.1, 1.0],
        'max_depth': [3, 5, 7],
    }

elif mdl == "ABR":
    from sklearn.ensemble import AdaBoostRegressor as abr

    # Define model
    model = abr()

    # Define parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150, 200, 300],
        'learning_rate': [0.01, 0.1, 1.0],
    }

elif mdl == "XGBR":
    from xgboost import XGBRegressor as xgboost

    # Define model
    model = xgboost()

    # Define parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150, 200, 300],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
    }

elif mdl == "KNN":
    from sklearn.neighbors import KNeighborsRegressor as knn

    # Define model
    model = knn()

    # Define parameter grid
    param_grid = {
        'n_neighbors': [3, 5, 7, 10, 15],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]  # p=1 is for Manhattan distance, p=2 is for Euclidean distance
    }

else:
    print("ERROR__" * 100)

# Define custom scorers for RMSE, MAE, R2
def custom_rmse_scorer(y_true, y_pred):
    return -np.sqrt(mean_squared_error(y_true, y_pred))

def custom_mae_scorer(y_true, y_pred):
    return -mean_absolute_error(y_true, y_pred)

def custom_r2_scorer(y_true, y_pred):
    return r2_score(y_true, y_pred)

r2_scorer = make_scorer(custom_r2_scorer)
rmse_scorer = make_scorer(custom_rmse_scorer)
mae_scorer = make_scorer(custom_mae_scorer)

# Create GridSearchCV objects
grid_search_rmse = GridSearchCV(model, param_grid, scoring=rmse_scorer, cv=5)
grid_search_rmse.fit(x_train, y_train)

# ----------------------------- RMSE ---------------------------------
print(mdl)
print("-" * 100)
print("RMSE Best Score: ", grid_search_rmse.best_score_)
print("RMSE Best Parameters: ", grid_search_rmse.best_params_)

grid_search_mae = GridSearchCV(model, param_grid, scoring=mae_scorer, cv=5)
grid_search_mae.fit(x_train, y_train)

# ----------------------------- MAE ---------------------------------
print("MAE Best Score: ", grid_search_mae.best_score_)
print("MAE Best Parameters: ", grid_search_mae.best_params_)

grid_search_r2 = GridSearchCV(model, param_grid, scoring=r2_scorer, cv=5)
grid_search_r2.fit(x_train, y_train)

# ----------------------------- R2 ---------------------------------
print("R2 Best Score: ", grid_search_r2.best_score_)
print("R2 Best Parameters: ", grid_search_r2.best_params_)


In [ ]:
# Get best RMSE parameters
#best_params = grid_search_rmse.best_params_
best_model = grid_search_rmse.best_estimator_
fac = "RMSE"
p_out = rmse_path
print("RMSE Best Score: ", grid_search_rmse.best_score_)
print("RMSE Best Parameters: ", grid_search_rmse.best_params_)
print(best_model)

In [ ]:
# Get best MAE parameters
#best_params = grid_search_rmse.best_params_
best_model = grid_search_mae.best_estimator_
fac = "MAE"
p_out = mae_path
print("MAE Best Score: ", grid_search_mae.best_score_)
print("MAE Best Parameters: ", grid_search_mae.best_params_)
print(best_model)

In [ ]:
# Get best r2 parameters
#best_params = grid_search_r2.best_params_
best_model = grid_search_r2.best_estimator_
fac = "r2"
p_out = r2_path
print("r2 Best Score: ", grid_search_r2.best_score_)
print("r2Best Parameters: ", grid_search_r2.best_params_)
print(best_model)

In [ ]:
#default
best_model = rfr()
fac = "DEF"
p_out = def_path

### Enter manualy when required

In [ ]:
#Manual 
#best_model = rfr()

### Fit regressor model and compute variable importance score 

may need to restrict the number of variables for the bar graph to be legible



In [ ]:
#tree_model = best_model.fit(X_1, y_1)
best_model.fit(x_train, y_train)

##### good info on the feature importance score - http://stackoverflow.com/questions/15810339/how-are-feature-importances-in-randomforestclassifier-determined

In [ ]:
from sklearn.inspection import permutation_importance
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import textwrap
from sklearn.metrics import mean_absolute_error

# Calculate feature importance for tree-based models
def tree_based_feature_importance(model, x_train, cols):
    fi = enumerate(model.feature_importances_)
    fiResult = [(value, cols[i]) for (i, value) in fi]
    return fiResult

# Calculate permutation importance
def permutation_feature_importance(model, x_train, y_train, cols):
    result = permutation_importance(model, x_train, y_train, n_repeats=10, random_state=42)
    fiResult = [(importance, cols[i]) for i, importance in enumerate(result.importances_mean)]
    return fiResult

# Plot feature importance
def plot_feature_importance(fiResult, model_name, var_, fac, mdl, output):
    df_band = pd.DataFrame(fiResult, columns=['importance', 'feature'])
    df_band['importance'] = df_band['importance'].astype(float)
    dfsort = df_band.sort_values(['importance'], ascending=[False]).head(20)  # Select top 20 features

    ind = np.arange(len(dfsort))
    width = 0.4

    # Increase figure size
    fig, ax = plt.subplots(figsize=(12, 8))

    # Bar plot
    ax.barh(ind, dfsort['importance'], width, color='blue')

    # Adjust y-tick labels
    wrapped_labels = [textwrap.fill(label, 20) for label in dfsort['feature']]
    ax.set(yticks=ind, yticklabels=wrapped_labels, ylim=[-1, len(dfsort)])
    ax.set_xlabel('Importance Score', fontsize=14)
    ax.set_ylabel('Features', fontsize=14)
    ax.set_title(f'Top 20 Feature Importance ({model_name})', fontsize=14)

    # Reverse the order of y-axis to display highest to lowest
    ax.invert_yaxis()

    # Adjust layout
    plt.tight_layout()

    out = os.path.join(output, f"{var_}_{fac}_{mdl}_Top20_Feature_Importance_Score.JPG")
    plt.savefig(out, dpi=300)  # dpi sets the resolution in dots per inch
    plt.show()
    print(out)

    return dfsort

# Feature importance
cols = xdata1.columns

if mdl in ["rfr", "abr", "gbr", "xgboost"]:
    fiResult = tree_based_feature_importance(best_model, x_train, cols)
else:
    fiResult = permutation_feature_importance(best_model, x_train, y_train, cols)

# Plot feature importance
dfsort = plot_feature_importance(fiResult, mdl, var_, fac, mdl, p_out)

# Generate scatter plot for model predictions
plt.figure(figsize=(10, 6))
y_pred = best_model.predict(x_train)

r2 = round(best_model.score(x_train, y_train), 2)
mse = round(np.mean((y_train - y_pred) ** 2), 2)
rmse = round(np.sqrt(mse), 2)
mae = round(mean_absolute_error(y_train, y_pred), 2)
bias = round(np.mean(y_train) - np.mean(y_pred), 2)

plt.scatter(y_pred, y_train, s=70, alpha=0.5, color='blue', edgecolors='w')

# Data for the 1 for 1 line
x = [-500, 40000]
y = [-500, 40000]

# Set the limits of the axis
plt.xlim(-500, 40000)
plt.ylim(-500, 40000)

plt.plot(x, y, color='red')

# Add labels and title
plt.ylabel('Observed Target', fontsize=14)
plt.xlabel('Predicted Target', fontsize=14)
plt.title(f'Data Used in {mdl} Training', fontsize=16)

# Annotate the stats in the top left corner
plt.text(0.05, 0.95, f'r2: {r2:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nBias: {bias:.2f}\nn: {len(y_train)}',
         transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='left', fontsize=12)

out = os.path.join(p_out, f"{var_}_{mdl}_{fac}_train.JPG")
plt.savefig(out, dpi=300)  # dpi sets the resolution in dots per inch
plt.show()

print("saved to: ", out)
print("-" * 30)
print(f"\t - r2: {r2}")
print(f"\t - mse: {mse}")
print(f"\t - rmse: {rmse}")
print(f"\t - mae: {mae}")
print(f"\t - bias: {bias}")
print(f"\t - n: {len(y_train)}")

# Example metrics
metrics_dict = {
    'r2': r2,
    'mse': mse,
    'rmse': rmse,
    'mae': mae,
    'bias': bias,
    'n': len(y_train)
}

# Adding the site as a key-value pair
metrics_dict['model'] = mdl
metrics_dict['status'] = "train"
metrics_dict['var'] = var_
metrics_dict['fac'] = fac
metrics_dict['features'] = list(x_train)

# Creating a DataFrame from the dictionary
train_metrics_df = pd.DataFrame([metrics_dict])

out_metrics = os.path.join(p_out, f"{var_}_{mdl}_{fac}_train_metrics.csv")
train_metrics_df.to_csv(out_metrics, index=False)

In [ ]:
train_metrics_df

In [ ]:
from sklearn.metrics import explained_variance_score, mean_absolute_error
import os

# Evaluate and plot for the best model
plt.figure(figsize=(10, 6))

# Use the best estimator from the grid search
#best_model = grid_search_r2.best_estimator_

# Make predictions
y2_predict = best_model.predict(x_test)

# Calculate metrics
r2 = round(best_model.score(x_test, y_test), 2)
mse = round(np.mean((y_test - y2_predict) ** 2), 2)
rmse = round(np.sqrt(mse), 2)
mae = round(mean_absolute_error(y_test, y2_predict), 2)
bias = round(np.mean(y_test) - np.mean(y2_predict), 2)
var = round(explained_variance_score(y_test, y2_predict), 2)

# Generate scatter plot
plt.scatter(y2_predict, y_test, s=70, alpha=0.5, color='blue', edgecolors='w')

# Data for the 1 for 1 line
x = [-500, 40000]
y = [-500, 40000]

# Set limits of the axis
plt.xlim(-500, 40000)
plt.ylim(-500, 40000)
plt.plot(x, y, color='red')

# Add labels and title
plt.ylabel('Observed Target', fontsize=14)
plt.xlabel('Predicted Target', fontsize=14)
plt.title(f'Data Used in {mdl} Testing', fontsize=16)

# Annotate the stats in the top left corner
plt.text(0.05, 0.95, f'r2: {r2:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nBias: {bias:.2f}\nn: {len(y_test)}',
         transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='left', fontsize=12)

# Save the plot
out_plot = os.path.join(p_out, f"{var_}_{mdl}_{fac}_test_data.JPG")
plt.savefig(out_plot, dpi=300)
plt.show()

print("Plot: ", out_plot)
print("-" * 30)
print(f"\t - r2: {r2}")
print(f"\t - mse: {mse}")
print(f"\t - rmse: {rmse}")
print(f"\t - mae: {mae}")
print(f"\t - bias: {bias}")
print(f"\t - n: {len(y_test)}")


# Example metrics
metrics_dict = {
    'r2': r2,
    'mse': mse,
    'rmse': rmse,
    'mae': mae,
    'bias': bias,
    'n': len(y_test)
}

# Adding the site as a key-value pair
metrics_dict['model'] = mdl
metrics_dict['status'] = "test"
metrics_dict['var'] = var_
metrics_dict['fac'] = fac
metrics_dict['features'] = list(x_test)

# Creating a DataFrame from the dictionary
test_metrics_df = pd.DataFrame([metrics_dict])

out_metrics = os.path.join(p_out, f"{var_}_{mdl}_{fac}_test_metrics.csv")
test_metrics_df.to_csv(out_metrics, index=False)



In [ ]:
test_metrics_df

In [ ]:
orig_dfsort = dfsort.copy()
#dfsort = orig_dfsort
#dfsort.shape

In [ ]:
dfsort

### Predict the selected model on the test data

In [ ]:
#Plot has value at the top with very small n value

sel_num = 20
df_var = dfsort.head(sel_num)

In [ ]:
#df_var = dfsort[dfsort['n'] > 0.01]

In [ ]:
df_var

In [ ]:
column_var = df_var.feature.tolist()

In [ ]:
#column_var.insert(0, 'target')

In [ ]:
column_var

In [ ]:
df_ml

In [ ]:
select_df = df_ml[column_var]

In [ ]:
df_corr = select_df.corr()
df_corr

In [ ]:
dfsort.to_csv(os.path.join(p_out, "{0}_{1}_{2}_sel_{3}_variable_score.csv".format(var_, fac, mdl, sel_num)))
df_corr.to_csv(os.path.join(p_out, "{0}_{1}_{2}_sel_{3}_variable_correlation.csv".format(var_, fac, mdl, sel_num)))

In [ ]:
# Create heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(df_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')

# Save the plot
plot_out = os.path.join(p_out, "{0}_{1}_{2}_sel_{3}_variable_score.JPG".format(var_, fac, mdl, sel_num))
plt.savefig(plot_out, dpi=300)  # dpi sets the resolution in dots per inch
plt.show()
print("Plot: ", plot_out)

In [ ]:
select_df

# Rerun on Selected data

In [ ]:
#x_train, x_test, y_train, y_test, x_validation, y_validation

In [ ]:
sel_x_train = x_train[column_var]
sel_x_test = x_test[column_var]
sel_x_validation = x_validation[column_var]
sel_y_train = y_train
sel_y_test = y_test
sel_y_validation = y_validation

# Set model with feature names for Notebook testing

In [ ]:
select_model = best_model.fit(sel_x_train, sel_y_train)

## Re-train model on selected paramiters

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import textwrap

# Calculate feature importance for tree-based models
def tree_based_feature_importance(model, x_train, cols):
    fi = enumerate(model.feature_importances_)
    fiResult = [(value, cols[i]) for (i, value) in fi]
    return fiResult

# Calculate permutation importance
def permutation_feature_importance(model, x_train, y_train, cols):
    result = permutation_importance(model, x_train, y_train, n_repeats=10, random_state=42)
    fiResult = [(importance, cols[i]) for i, importance in enumerate(result.importances_mean)]
    return fiResult

# Plot feature importance
def plot_feature_importance(fiResult, model_name, var_, fac, mdl, output):
    df_band = pd.DataFrame(fiResult, columns=['importance', 'feature'])
    df_band['importance'] = df_band['importance'].astype(float)
    dfsort = df_band.sort_values(['importance'], ascending=[False]).head(20)  # Select top 20 features

    ind = np.arange(len(dfsort))
    width = 0.4

    # Increase figure size
    fig, ax = plt.subplots(figsize=(12, 8))

    # Bar plot
    ax.barh(ind, dfsort['importance'], width, color='blue')

    # Adjust y-tick labels
    wrapped_labels = [textwrap.fill(label, 16) for label in dfsort['feature']]
    ax.set(yticks=ind, yticklabels=wrapped_labels, ylim=[-1, len(dfsort)])
    ax.set_xlabel('Importance Score', fontsize=12)
    ax.set_ylabel('Features', fontsize=12)
    ax.set_title(f'Top {sel_num} Feature Importance ({mdl})', fontsize=14)

    # Reverse the order of y-axis to display highest to lowest
    ax.invert_yaxis()

    # Adjust layout
    plt.tight_layout()

    out = os.path.join(p_out, f"{var_}_{fac}_{mdl}_Top_{sel_num}_Feature_Importance_Score.JPG")
    plt.savefig(out, dpi=300)  # dpi sets the resolution in dots per inch
    plt.show()
    print(out)

    return dfsort

# Feature importance
cols = xdata1.columns

if isinstance(select_model, (rfr, abr, gbr)):
    fiResult = tree_based_feature_importance(select_model, sel_x_train, cols)
else:
    fiResult = permutation_feature_importance(select_model, sel_x_train, sel_y_train, cols)

# Plot feature importance
dfsort = plot_feature_importance(fiResult, mdl, var_, fac, mdl, p_out)

# Generate scatter plot for model predictions
plt.figure(figsize=(10, 6))
sel_y_pred = select_model.predict(sel_x_train)

r2 = round(select_model.score(sel_x_train, sel_y_train), 2)
mse = round(np.mean((sel_y_train - sel_y_pred) ** 2), 2)
rmse = round(np.sqrt(mse), 2)
mae = round(mean_absolute_error(sel_y_train, sel_y_pred), 2)
bias = round(np.mean(sel_y_train) - np.mean(sel_y_pred), 2)

plt.scatter(sel_y_pred, sel_y_train, s=70, alpha=0.5, color='blue', edgecolors='w')

# Data for the 1 for 1 line
x = [-500, 40000]
y = [-500, 40000]

# Set the limits of the axis
plt.xlim(-500, 40000)
plt.ylim(-500, 40000)

plt.plot(x, y, color='red')

# Add labels and title
plt.ylabel('Observed Target', fontsize=14)
plt.xlabel('Predicted Target', fontsize=14)
plt.title(f'Data Used in {mdl} Training', fontsize=16)

# Annotate the stats in the top left corner
plt.text(0.05, 0.95, f'r2: {r2:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nBias: {bias:.2f}\nn: {len(sel_y_train)}',
         transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='left', fontsize=12)

out = os.path.join(output, f"{var_}_{mdl}_{fac}_top_{sel_num}_retrain.JPG")
plt.savefig(out, dpi=300)  # dpi sets the resolution in dots per inch
plt.show()

print("saved to: ", out)
print("-" * 30)
print(f"\t - r2: {r2}")
print(f"\t - mse: {mse}")
print(f"\t - rmse: {rmse}")
print(f"\t - mae: {mae}")
print(f"\t - bias: {bias}")
print(f"\t - n: {len(sel_y_train)}")


# Example metrics
metrics_dict = {
    'r2': r2,
    'mse': mse,
    'rmse': rmse,
    'mae': mae,
    'bias': bias,
    'n': len(sel_y_train)
}

# Adding the site as a key-value pair
metrics_dict['model'] = mdl
metrics_dict['status'] = "retrain"
metrics_dict['var'] = var_
metrics_dict['fac'] = fac
metrics_dict['sel_num'] = sel_num
metrics_dict['features'] = list(sel_x_train)

# Creating a DataFrame from the dictionary
retrain_metrics_df = pd.DataFrame([metrics_dict])

out_metrics = os.path.join(p_out, f"{var_}_{mdl}_{fac}_top_{sel_num}_retrain_metrics.csv")
retrain_metrics_df.to_csv(out_metrics, index=False)


In [ ]:
plt.figure(figsize=(10, 6))
y2_predict = select_model.predict(sel_x_test)

r2 = round(select_model.score(sel_x_test, sel_y_test), 2)
mse = round(np.mean((sel_y_test - select_model.predict(sel_x_test)) ** 2), 2)
rmse = round(np.sqrt(np.mean((y2_predict - sel_y_test) ** 2)), 2)
mae = round(mean_absolute_error(sel_y_test, y2_predict), 2)
bias = round(np.mean(sel_y_test) - np.mean(y2_predict), 2)
var = round(explained_variance_score(sel_y_test, y2_predict), 2)

plt.scatter(y2_predict, sel_y_test, s=70, alpha=0.5, color='blue', edgecolors='w')

# Data for the 1 for 1 line
x = [-500, 40000]
y = [-500, 40000]

# Set the limits of the axis
plt.xlim(-500, 40000)
plt.ylim(-500, 40000)
plt.plot(x, y, color='red')

# Add labels and title
plt.ylabel('Observed Target', fontsize=14)
plt.xlabel('Predicted Target', fontsize=14)
plt.title(f'Data Used in {mdl} Testing', fontsize=16)

# Annotate the stats in the top left corner
plt.text(0.05, 0.95, f'r2: {r2:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nBias: {bias:.2f}\nn: {len(sel_y_test)}',
         transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='left', fontsize=12)

# Save the plot
plt.savefig(os.path.join(output, f"{var_}_{mdl}_{fac}_top_{sel_num}_retest.JPG"), dpi=300)
plt.show()

# Print the results
print("-" * 30)
print(f"\t - r2: {r2}")
print(f"\t - mse: {mse}")
print(f"\t - rmse: {rmse}")
print(f"\t - mae: {mae}")
print(f"\t - bias: {bias}")
print(f"\t - n: {len(sel_y_test)}")


# Example metrics
metrics_dict = {
    'r2': r2,
    'mse': mse,
    'rmse': rmse,
    'mae': mae,
    'bias': bias,
    'n': len(y_test)
}

# Adding the site as a key-value pair
metrics_dict['model'] = mdl
metrics_dict['status'] = "retest"
metrics_dict['var'] = var_
metrics_dict['fac'] = fac
metrics_dict['sel_num'] = sel_num
metrics_dict['features'] = list(sel_x_test)

# Creating a DataFrame from the dictionary
retest_metrics_df = pd.DataFrame([metrics_dict])

out_metrics = os.path.join(p_out, f"{var_}_{mdl}_{fac}_top_{sel_num}_metrics_retest.csv")
retest_metrics_df.to_csv(out_metrics, index=False)



In [ ]:
variable_imp_list = dfsort.feature.to_list()
print(variable_imp_list)

# =================================================================

# STOP - do you realy want to save this model?

# remember to change the cPickle file name !!

# Save current fitted model and apply to validation validation data

In [ ]:
select_model

In [ ]:
import pickle

#rfrL8CHM = rfr()
#rfrL8CHM.fit(sel_x_train, sel_y_train)
pickle_file = os.path.join(output, "{0}_{1}_{2}_sel_{3}_{4}_model.pickle".format(var_, fac, mdl, sel_num, samp))
with open(pickle_file, 'wb') as f:
    pickle.dump(select_model, f)
print("pickle saved: ", pickle_file)

In [ ]:
# read in your validation dataset which has never been seen by rfr model - NOTE in this example I am just reading the same data used to train the model
sel_y_validation
sel_x_validation
#validation_data

In [ ]:
c_list = sel_x_validation.columns.tolist()
c_list

In [ ]:
validation_df = sel_x_validation

In [ ]:
# Insert array as the first column
validation_df.insert(0, 'target', sel_y_validation)

In [ ]:
validation_out = os.path.join(output, "validation_data.csv")
validation_df.to_csv(validation_out, index=False)

In [ ]:
#select_validation_df = validation_df[column_var]

In [ ]:
validation_df.columns

In [ ]:
column_var

In [ ]:
validation_xdata = validation_df.iloc[:, 1:].astype('float32')
ydata1 = validation_df[[value_x]].astype('float32')
ydata2 = ydata1.values
ydata3 = ydata2.ravel()

validation_ydata = ydata3.reshape(-1, 1)

In [ ]:
validation_xdata.shape

In [ ]:
validation_ydata.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import explained_variance_score, mean_absolute_error

plt.figure(figsize=(10, 6))

# Load the model
with open(pickle_file, 'rb') as f:
    rf = pickle.load(f)

predicted = rf.predict(validation_xdata)

# Calculate metrics
r2 = round(rf.score(validation_xdata, validation_ydata), 2)
mse = round(np.mean((validation_ydata - predicted) ** 2), 2)
rmse = round(np.sqrt(np.mean((predicted - validation_ydata) ** 2)), 2)
mae = round(mean_absolute_error(validation_ydata, predicted), 2)
bias = round(np.mean(validation_ydata) - np.mean(predicted), 2)
var = round(explained_variance_score(validation_ydata, predicted), 2)

# Plot predicted vs observed data
plt.scatter(predicted, validation_ydata, s=70, alpha=0.5, color='blue', edgecolors='w')

# Data for the 1-for-1 line
x = [-500, 40000]
y = [-500, 40000]

# Set the limits of the axis
plt.xlim(-500, 40000)
plt.ylim(-500, 40000)

# Add labels and title
plt.ylabel('Observed Target', fontsize=14)
plt.xlabel('Predicted Target', fontsize=14)
plt.title(f'Data Used in {mdl} Validation', fontsize=16)

# Annotate the stats in the top left corner
plt.text(0.05, 0.95, f'r2: {r2:.2f}\nMSE: {mse:.2f}\nRMSE: {rmse:.2f}\nMAE: {mae:.2f}\nBias: {bias:.2f}\nn: {len(validation_ydata)}',
         transform=plt.gca().transAxes, verticalalignment='top', horizontalalignment='left', fontsize=12)

# 1-for-1 line
plt.plot(x, y, color='red')

# Save the plot
output_plot = os.path.join(output, "{0}_{1}_{2}_sel_{3}_data_validation.JPG".format(var_, fac, mdl, str(sel_num)))
plt.savefig(output_plot, dpi=600)
plt.show()

# Print the results
print("plot output: ", output_plot)
print("Using: ", "{0}_{1}_{2}_sel_{3}_model.pickle".format(var_, fac, mdl, sel_num))

print("-" * 30)
print(f"\t - r2: {r2}")
print(f"\t - mse: {mse}")
print(f"\t - rmse: {rmse}")
print(f"\t - mae: {mae}")
print(f"\t - bias: {bias}")
print(f"\t - n: {len(validation_ydata)}")



# Adding the site as a key-value pair
metrics_dict['model'] = mdl
metrics_dict['status'] = "validate"
metrics_dict['var'] = var_
metrics_dict['fac'] = fac
metrics_dict['sel_num'] = sel_num
metrics_dict['features'] = list(validation_df)

# Creating a DataFrame from the dictionary
val_metrics_df = pd.DataFrame([metrics_dict])


out_metrics = os.path.join(output, f"{var_}_{mdl}_{fac}_top_{sel_num}_validate_metrics.csv")
val_metrics_df.to_csv(out_metrics, index=False)

In [ ]:
# Load the pickled model
import pickle

with open(pickle_file, 'rb') as file:
    model = pickle.load(file)

# Load the dataset
import pandas as pd

# validation x data
data = validation_xdata

# Check the feature names used during training
training_feature_names = model.feature_names_in_

# Check the feature names in the new dataset
new_feature_names = data.columns

print("Training feature names:", training_feature_names)
print("New feature names:", new_feature_names)

# Adjust to export model with no feature names

In [ ]:
xarray = sel_x_train.to_numpy()
yarray = sel_y_train

In [ ]:
select_tree_model_no_headers = best_model.fit(xarray, yarray)

## Export selected model as pickle

In [ ]:
import pickle

#rfrL8CHM = rfr()
#rfrL8CHM.fit(sel_x_train, sel_y_train)
pickle_file_no_headers = os.path.join(output, "{0}_{1}_{2}_sel_{3}_{4}_model_no_headers.pickle".format(var_, fac, mdl, sel_num, samp))
with open(pickle_file_no_headers, 'wb') as f:
    pickle.dump(select_tree_model_no_headers, f)
print("pickle saved: ", pickle_file_no_headers)

In [ ]:
plt.close('all')